## Create our types

First create an Abstract type `Temperature` (https://docs.julialang.org/en/v1/manual/types/#Abstract-Types-1)
and types `Celsius`, `Kelvin` and `Fahrenheit` that herit from `Temperature` and have a float64 `value` attribute

In [1]:
abstract type Temperature end

struct Celsius <: Temperature
    value::Float64
end

struct Kelvin <: Temperature
   value::Float64 
end

struct Fahrenheit <: Temperature
    value::Float64
end

Bonus: associate the temperature symbols with the unit value of each unit measure.

In [2]:
const °C = Celsius(1)
const °F = Fahrenheit(1)
const K = Kelvin(1)

Kelvin(1.0)

 Convert Kelvin to Celsius (kelvin = celsius - 273.15), Celsius to Kelvin, Celsius to Fahrenheit (Fahrenheit=Celsius*9/5 + 32) and Fahrenheit to Celsius. You will need to import the `convert` function from Base

In [3]:
import Base: convert # note what happened if we defined convert before

convert(::Type{Kelvin},  celsius::Celsius)        = Kelvin(celsius.value + 273.15)
convert(::Type{Celsius}, kelvin::Kelvin)          = Celsius(kelvin.value - 273.15)
convert(::Type{Celsius}, fahrenheit::Fahrenheit)  = Celsius((fahrenheit.value - 32)*5/9)
convert(::Type{Fahrenheit}, celsius::Celsius)     = Fahrenheit(celsius.value*9/5 + 32)

convert (generic function with 190 methods)

Then add also Kelvin to Farhenheit and Farhenheit to Kelvin

In [4]:
convert(::Type{Fahrenheit}, kelvin::Kelvin) = Fahrenheit(Celius(kelvin))
convert(::Type{Kelvin},  fahrenheit::Fahrenheit)  = Kelvin(Celsius(fahrenheit))

convert (generic function with 192 methods)

## New constructors
Create constructors `Kelvin(Celsius)`, `Celsius(Kelvin)`...

In [5]:
Kelvin(celsius::Celsius)         = convert(Kelvin,  celsius)
Celsius(kelvin::Kelvin)          = convert(Celsius, kelvin)
Celsius(fahrenheit::Fahrenheit)  = convert(Celsius, fahrenheit)
Fahrenheit(celsius::Celsius) = convert(Fahrenheit, celsius) 

Fahrenheit

Add `+,-` operators between two values of same unit

In [6]:
import Base: +,-, isapprox
+(x::T, y::T) where {T <: Temperature} = T(x.value + y.value)
-(x::T, y::T) where {T <: Temperature} = T(x.value - y.value)
isapprox(x::T, y::T) where {T <: Temperature} = isapprox(x.value, y.value)

*(x::Number, y::T) where {T <: Temperature} = T(x * y.value)

* (generic function with 1 method)

## Manage mix units

Define your heriarchy of temperature : what happens when we have Kelvin and Celsius, Celsius and Fahrenheit ?
For that use promote_rule from Base
Here we choose to promote 
- Kelvin + Celsius -> Kelvin
- Fahrenheit + Celsius -> Celsius
- Fahrenheit + Kelvin -> Kelvin
Use promote and promote_rule from Base to manage the operations on mix units

In [7]:
import Base: promote_rule

promote_rule(::Type{Kelvin},     ::Type{Celsius}) = Kelvin
promote_rule(::Type{Fahrenheit}, ::Type{Celsius}) = Celsius
promote_rule(::Type{Fahrenheit}, ::Type{Kelvin}) = Kelvin

promote_rule (generic function with 139 methods)

In [8]:
+(x::Temperature, y::Temperature) = +(promote(x,y)...)
-(x::Temperature, y::Temperature) = -(promote(x,y)...)
isapprox(x::Temperature, y::Temperature) = isapprox(promote(x,y)...)

isapprox (generic function with 10 methods)

You can now see https://github.com/ordovician/Airship.jl, and https://github.com/ajkeller34/Unitful.jl